In [14]:
# Import statements go here
from fastapi import FastAPI, responses
from pydantic import BaseModel
import pickle
from configparser import ConfigParser
import numpy as np
import pandas as pd

from embedders.classification.contextual import TransformerSentenceEmbedder

In [16]:
df = pd.read_csv('/home/leonardpuettmann/repos/automl-docker/sample_data/final_export.csv')

In [17]:
df.columns

Index(['Unnamed: 0', 'running_id', 'link', 'article_type', 'headline',
       'content_extract', 'author', 'publication_time', 'shares',
       '__final verdict__MANUAL', '__final verdict__WEAK_SUPERVISION',
       '__final verdict__WEAK_SUPERVISION__confidence', 'combined'],
      dtype='object')

In [23]:
print(df.headline[7])
print(df.content_extract[7])
print(df['__final verdict__WEAK_SUPERVISION'][7])

Auf der Suche nach extremistischen Beamten
Heute jährt sich eine der umstrittensten Entscheidungen der Ministerpräsidentenkonferenz: Wie soll der Staat mit Beamten und Anwärtern umgehen, an deren Verfassungstreue es Zweifel gibt?
0


In [24]:

config = ConfigParser()
config.read('/home/leonardpuettmann/repos/automl-docker/ml/config.ini')
model = config['Transformer_Model']['model_used']
transformer = TransformerSentenceEmbedder(model)

corpus = ['Endstation Verzweiflung bei der Deutschen Bahn Die Bahn steckt fest im selbst verschuldeten Schlamassel aus zahlreichen Baustellen und endlosen Verspätungen. Findet sie da jemals wieder raus?', 'Auf der Suche nach extremistischen Beamten Auf der Suche nach extremistischen Beamten \
Heute jährt sich eine der umstrittensten Entscheidungen der Ministerpräsidentenkonferenz: Wie soll der Staat mit Beamten und Anwärtern umgehen, an deren Verfassungstreue es Zweifel gibt?']

    # Create embeddings to make text usable by ml model
embeddings = transformer.transform(corpus)


Some weights of the model checkpoint at /home/leonardpuettmann/.cache/torch/sentence_transformers/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initializing model, might take some time...


Encoding batches ...: 100%|██████████| 1/1 [00:00<00:00,  4.90it/s]


In [25]:
# Use ml model to create predictions
model = pickle.load(open("/home/leonardpuettmann/repos/automl-docker/ml/model.pkl", "rb"))
predictions = model.predict(embeddings).tolist()
probabilities = model.predict_proba(embeddings)
probabilities_max = np.max(probabilities, axis=1).tolist()
probabilities_rounded = [round(prob, 2) for prob in probabilities_max]
probabilities_pct = [prob * 100 for prob in probabilities_rounded]

In [26]:
dictionary = dict(zip(probabilities_pct, predictions))
print(dictionary)


{81.0: 1, 71.0: 0}
